In [7]:
import numpy as np
import struct
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from torchvision import datasets, transforms
import time
import statistics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D

In [6]:
mnist_training, mnist_testing = tf.keras.datasets.mnist.load_data()
training_data = np.float16(mnist_training[0])  # Returns np.array
training_labels = np.asarray(mnist_training[1], dtype=np.int32)
testing_data = np.float16(mnist_testing[0])  # Returns np.array
testing_labels = np.asarray(mnist_testing[1], dtype=np.int32)

In [16]:
feature_columns = [tf.feature_column.numeric_column("images", shape=[28, 28])]

train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"images": training_data},
    y= training_labels,
    num_epochs = None,
    batch_size = 50,
    shuffle = True
)

test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x = {"images": testing_data},
    y = testing_labels,
    num_epochs = 1,
    shuffle = False
)

In [14]:
def repeatedTrainingAndTesting(model_name, dropout, batch_normalization, M):
    
    if batch_normalization and not(dropout):
        return
    # baeldung.com/cs/batch-normalization-cnn
        
    elif not(batch_normalization) and dropout:
        return
        
    elif not(batch_normalization) and not(dropout):
        model = Sequential([
            Conv2D(60000, (28, 28), activation='relu', input_shape=(60000, 28, 28))
        ]);
        
    #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        
    elif batch_normalization and dropout:
        return
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    
    accuracies = []
    totalTime = 0
    
    for i in range(M):
        
        print('_______________________________________________________________________ M='+str(i+1))
        
        # TRAIN
        start_time = time.time()
        model.fit(training_data, training_labels, epochs=10)
        end_time = time.time()
        duration = end_time - start_time
        print("Training time: --- %s seconds ---" % (duration))
        totalTime += duration
        
        # TEST
        start_time = time.time()
        accuracy_score = model.evaluate(testing_data, testing_labels)[1]
        end_time = time.time()
        duration = end_time - start_time
        print("Testing Accuracy: {0:f}%\n".format(accuracy_score*100))
        print("Testing time: --- %s seconds ---" % (duration))
        accuracies.append(accuracy_score)
        totalTime += duration
        
    return (accuracies, totalTime)

# MLP

In [12]:
m = 1

## With Dropout, With Batch-Normalization

In [ ]:
(accuracies_DO_BN, time_DO_BN) = repeatedTrainingAndTesting(model_name = "cnn_DO_BN", dropout = 0.2, batch_normalization = True, M=m)

# Without Dropout, With Batch-Normalization

In [ ]:
(accuracies_BN, time_BN) = repeatedTrainingAndTesting(model_name = "cnn_BN", dropout = None, batch_normalization = True, M=m)

# With Dropout, Without Batch-Normalization

In [ ]:
(accuracies_DO, time_DO) = repeatedTrainingAndTesting(model_name = "cnn_DO", dropout = 0.2, batch_normalization = False, M=m)

# Without Dropout, Without Batch-Normalization

In [15]:
(accuracies_none, time_none) = repeatedTrainingAndTesting(model_name = "cnn_none", dropout = None, batch_normalization = False, M=m)

KeyboardInterrupt: 

# Save Results

In [ ]:
data = {'Accuracy_DO_BN': accuracies_DO_BN, 'Time_DO_BN': time_DO_BN } , 
        'Accuracy_BN' : accuracies_BN, 'Time_BN' : time_BN, 
        'Accuracy_DO' : accuracies_DO, 'Time_DO': time_DO, 
        'Accuracy_NONE': accuracies_none, 'Time_NONE': time_none}

df = pd.DataFrame(data)

df.to_csv('outputs/CNN.csv')